This notebook aims to build a graph purely with KNN.

In [1]:
import numpy as np
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-trainval', dataroot='/media/HDD2/Datasets/mini_nusc', verbose=True)

# nusc = NuScenes(version='v1.0-trainval', dataroot='/media/HDD2/Datasets/nuscenes2', verbose=True)

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.649 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [2]:
import torch_geometric as tgeo

tgeo.get_home_dir()

'/home/maximilian/.cache/pyg'

In [3]:
# #Get set of scenes
# scenes = nusc.scene
# #Get first scenes
# scene_0 = scenes[0]
# # Get token of first frame
# first_sample_token = scene_0['first_sample_token']

# NUMBER_OF_SKIPPED_FRAMES = 10
# for i in range(NUMBER_OF_SKIPPED_FRAMES):
#     temp_sample = nusc.get('sample', first_sample_token)
#     temp_token = temp_sample['next']
#     first_sample_token = temp_token

# sample_0 = nusc.get('sample', first_sample_token)
# # Get tokens for 2 following frames
# second_sample_token = sample_0['next']
# sample_1 = nusc.get('sample', second_sample_token)
# third_sample_token = sample_1['next']
# sample_2 = nusc.get('sample', third_sample_token)

# # Get LIDAR pointcloud
# sensor = 'LIDAR_TOP'
# lidar_top_data_0 = nusc.get('sample_data', sample_0['data'][sensor])
# # Get LIDAR KF pointcloud
# lidar_top_data_1 = nusc.get('sample_data', sample_1['data'][sensor])
# # Get LIDAR KF pointcloud
# lidar_top_data_2 = nusc.get('sample_data', sample_2['data'][sensor])

# #Filter out car/vehicle object
# pcl0_path, boxes0, _= nusc.get_sample_data(lidar_top_data_0['token'], selected_anntokens=None, use_flat_vehicle_coordinates =False)
# pcl1_path, boxes1, _= nusc.get_sample_data(lidar_top_data_1['token'], selected_anntokens=None, use_flat_vehicle_coordinates =False)
# pcl2_path, boxes2, _= nusc.get_sample_data(lidar_top_data_2['token'], selected_anntokens=None, use_flat_vehicle_coordinates =False)


In [4]:
# # Get point clouds
# import os.path as osp
# from nuscenes.utils.data_classes import LidarPointCloud, Box

# #Load Pointclouds
# pc0 = LidarPointCloud.from_file(pcl0_path)
# pc1 = LidarPointCloud.from_file(pcl1_path)
# pc2 = LidarPointCloud.from_file(pcl2_path)

# from utility import get_box_centers, filter_boxes

# car_boxes_0 = filter_boxes(nusc, boxes = boxes0, categoryQuery= 'vehicle.car')
# car_boxes_1 = filter_boxes(nusc, boxes = boxes1, categoryQuery= 'vehicle.car')
# car_boxes_2 = filter_boxes(nusc, boxes = boxes2, categoryQuery= 'vehicle.car')

# centers0 = get_box_centers(car_boxes_0)
# centers1 = get_box_centers(car_boxes_1)
# centers2 = get_box_centers(car_boxes_2)

# # Special Shift parameter in meter
# SPATIAL_SHIFT_TIMEFRAMES = 20
# # Boxes 0 can stay at the current frame
# # centers0
# # Boxes 1 must be translated up by l meters
# centers1 += np.array([0,0,SPATIAL_SHIFT_TIMEFRAMES])

# # Boxes 2 must be translated up by 2*l meters
# centers2 += np.array([0,0,2*SPATIAL_SHIFT_TIMEFRAMES])

# # Add all centroids into one array
# centers = np.empty((0,3))
# centers = np.append(centers, centers0, axis=0)
# centers = np.append(centers, centers1, axis=0)
# centers = np.append(centers, centers2, axis=0)

In [5]:
# # Build KNN Graph
# from sklearn.neighbors import NearestNeighbors
# from graph.graph_generation import Graph

# # Add temporal edges / Make connections#####################
# temporal_pointpairs = []

# TEMPORAL_KNN_PARAM = 3
# ''' 
# Defines the number of searched neighbors
# '''

# # connect frame-0-nodes with frame-1-nodes
# for i in range(len(centers0)):
#     center = centers0[i]
#     center = np.expand_dims(center,axis=0)
#     temp = np.append(centers1,center,axis=0)
#     #Find nearest_neigbor
#     nearest_neigbor = NearestNeighbors(n_neighbors=TEMPORAL_KNN_PARAM, algorithm='ball_tree').fit(temp)
#     temporal_distances, temporal_indices = nearest_neigbor.kneighbors(temp)
#     #Add indices into a list
#     for index in temporal_indices[-1]:
#         #adapt the index to the global indexing
#         # temporal_pointpairs.append([i, index + len(centers0)])

#         # find global indices and append them
#         reference_node_global_index = np.argwhere(centers == center)[0,0]
#         neighbor_node_global_index = np.argwhere(centers == temp[index])[0,0] 
#         temporal_pointpairs.append([reference_node_global_index ,\
#              neighbor_node_global_index ])

# # connect frame-0-nodes with frame-2-nodes
# for i in range(len(centers0)):
#     center = centers0[i]
#     center = np.expand_dims(center,axis=0)
#     temp = np.append(centers2,center,axis=0)
#     #Find nearest_neigbor
#     nearest_neigbor = NearestNeighbors(n_neighbors=TEMPORAL_KNN_PARAM, algorithm='ball_tree').fit(temp)
#     temporal_distances, temporal_indices = nearest_neigbor.kneighbors(temp)
#     #Add indices into a list (The last entry belongs to center!)
#     for index in temporal_indices[-1]:
#         #adapt the index to the global indexing
#         # temporal_pointpairs.append([i, index + len(centers0)])

#         # find global indices and append them
#         reference_node_global_index = np.argwhere(centers == center)[0,0]
#         neighbor_node_global_index = np.argwhere(centers == temp[index])[0,0] 
#         temporal_pointpairs.append([reference_node_global_index ,\
#              neighbor_node_global_index ])

# # connect frame-1-nodes with frame-2-nodes
# for i in range(len(centers1)):
#     center = centers1[i]
#     center = np.expand_dims(center,axis=0)
#     temp = np.append(centers2,center,axis=0)
#     nearest_neigbor = NearestNeighbors(n_neighbors=TEMPORAL_KNN_PARAM, algorithm='ball_tree').fit(temp)
#     temporal_distances, temporal_indices = nearest_neigbor.kneighbors(temp)

#     # Test if the last input is the appended center point
#     # assert (temporal_distances[-1] == temporal_distances[np.argwhere(temp == center)[0,0]]).all()

#     for index in temporal_indices[-1]:
#         #adapt the index to the global indexing
#         # temporal_pointpairs.append([i + len(centers0) , index + len(centers0) + len(centers1) ])
        
#         # find global indices and append them
#         reference_node_global_index = np.argwhere(centers == center)[0,0]
#         neighbor_node_global_index = np.argwhere(centers == temp[index])[0,0] 
#         temporal_pointpairs.append([reference_node_global_index ,\
#              neighbor_node_global_index ])


In [6]:
# #Build graph in a more organised manner
# from graph.graph_generation import SpatioTemporalGraph, Timeframe

# SPATIAL_NEIGHBOR_NUMS = 5

# nbrs_0 = NearestNeighbors(n_neighbors=SPATIAL_NEIGHBOR_NUMS, algorithm='ball_tree').fit(centers0)
# # Frame t0
# #Compute K nearest neighbors
# spatial_distances_0, spatial_indices_0 = nbrs_0.kneighbors(centers0)
# # Make a list of tuple pairs
# spatial_pairs = [] 
# for curr_node_idx ,neigborhood_indices,  in enumerate(spatial_indices_0):
#     for neigbor_index in neigborhood_indices:
#         spatial_pairs.append( \
#             ( (Timeframe.t0,curr_node_idx) , ((Timeframe.t0,neigbor_index)) ) )

# #Frame t1
# nbrs_1 = NearestNeighbors(n_neighbors=SPATIAL_NEIGHBOR_NUMS, algorithm='ball_tree').fit(centers1)
# spatial_distances_1, spatial_indices_1 = nbrs_1.kneighbors(centers1)
# # Make a list of tuple pairs
# for curr_node_idx ,neigborhood_indices,  in enumerate(spatial_indices_1):
#     for neigbor_index in neigborhood_indices:
#         spatial_pairs.append( \
#             ( (Timeframe.t1,curr_node_idx) , ((Timeframe.t1,neigbor_index)) ) )

# #Frame t2
# nbrs_2 = NearestNeighbors(n_neighbors=SPATIAL_NEIGHBOR_NUMS, algorithm='ball_tree').fit(centers2)
# spatial_distances_2, spatial_indices_2 = nbrs_2.kneighbors(centers2)
# # Make a list of tuple pairs
# for curr_node_idx ,neigborhood_indices,  in enumerate(spatial_indices_2):
#     for neigbor_index in neigborhood_indices:
#         spatial_pairs.append( \
#             ( (Timeframe.t2,curr_node_idx) , ((Timeframe.t2,neigbor_index)) ) )

# testgraph = SpatioTemporalGraph(spatial_pairs)

# # Get all spatial Edges
# spatial_pointpairs0 = []
# for reference_node in testgraph._graph:
#     if(reference_node[0]== Timeframe.t0):
#         for neighbor_node in testgraph._graph[reference_node]:
#             # print(neighbor_index[0])
#             timestep, idx = neighbor_node[0],neighbor_node[1]
#             if timestep == Timeframe.t0:
#                 spatial_pointpairs0.append([reference_node[1],idx])

# print(np.shape(spatial_pointpairs0))
# testarray = testgraph.get_spatial_pointpairs(Timeframe.t0)
# assert spatial_pointpairs0 == testarray

# spatial_pointpairs1 = testgraph.get_spatial_pointpairs(Timeframe.t1)
# spatial_pointpairs2 = testgraph.get_spatial_pointpairs(Timeframe.t2)

In [7]:
# # Generate  Flow Labels for TrackingGNN
# from groundtruth_generation.nuscenes_create_gt import generate_flow_labels
# from utils.nuscenes_helper_functions import is_valid_box

# car_boxes = [] 
# for list in [car_boxes_0, car_boxes_1, car_boxes_2]:
#     for box in list:
#         car_boxes.append(box)

# if len(car_boxes)== len(centers):
#     print('length', len(car_boxes))
#     # for box, center in zip(car_boxes, centers):
#     for i in range(len(car_boxes)):
#         box, center = car_boxes[i], centers[i]
#         if not is_valid_box(box, center, num_frames = 3, spatial_shift_timeframes=SPATIAL_SHIFT_TIMEFRAMES):
#             print('invalid boxes at ',i)
#             print('box',box.center)
#             print('center',center)

# flow_labels = generate_flow_labels(nusc,temporal_pointpairs, car_boxes, centers)


In [8]:
# from torch_geometric.data import Data
# from torch_geometric.loader import DataLoader
# import torch
# edge_index = torch.tensor([[0, 1, 1, 2],
#                            [1, 0, 2, 1]], dtype=torch.long)
# x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

# data = Data(x=x, edge_index=edge_index)


In [9]:
# # Create list of Data elements
# # 
# data_list = []
# spatio_temporal_edge_indices = temporal_pointpairs \
#         + spatial_pointpairs0

# data_list.append(Data(x= centers,edge_index= spatio_temporal_edge_indices, y= flow_labels))

# standard_loader = DataLoader(data_list, batch_size=1)

In [10]:
# from train import start_training

# batch_size = 1
# num_epochs = 5
# start_training(standard_loader, batch_size, num_epochs)


In [11]:
from train import start_training
from datasets.nuscenes_dataset import NuscenesDataset

# train_dataset = NuscenesDataset()
# batch_size = 1
# num_epochs = 5
# start_training(train_dataset, batch_size, num_epochs)

In [12]:
from nuscenes.utils.splits import train,val
from nuscenes.utils.splits import create_splits_scenes

split = create_splits_scenes()
print(split.keys())
split_scene_list = []
for scene_name in split['mini_train']:
    for scene in nusc.scene:
        if scene['name']==scene_name:
            split_scene_list.append(scene)

dict_keys(['train', 'val', 'test', 'mini_train', 'mini_val', 'train_detect', 'train_track'])


In [13]:
#write a sequence 

sample_dict = {}

i = 0 
for scene in split_scene_list:
    last_sample_token =""
    sample_token = scene['first_sample_token']
    while(last_sample_token == ""):
        
        sample = nusc.get('sample', sample_token)
        sample_dict[i] = (scene['token'],sample["token"])
        i += 1
        sample_token = sample["next"]
        if(sample["token"]== scene['last_sample_token']):
            last_sample_token = scene['last_sample_token']

print (len(sample_dict))
print (sample_dict[0])
print (sample_dict[1])
print (sample_dict[2])

323
('cc8c0bf57f984915a77078b10eb33198', 'ca9a282c9e77460f8360f564131a8af5')
('cc8c0bf57f984915a77078b10eb33198', '39586f9d59004284a7114a68825e8eec')
('cc8c0bf57f984915a77078b10eb33198', '356d81f38dd9473ba590f39e266f54e5')


In [14]:
import torch
# # setting device on GPU if available, else CPU
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print('Using device:', device)

# #Additional Info when using cuda
# if device.type == 'cuda':
#     print(torch.cuda.is_available())
#     num_gpu = torch.cuda.device_count()
#     print("num_gpu:",num_gpu)
#     print("current_device: ",torch.cuda.current_device())
#     print("---------------------------------------------------------------")
#     for i in range(num_gpu):
#         print(torch.cuda.get_device_name(i))
#         # print(torch.cuda.device(i))
#         print('Memory Usage:')
#         print('Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB', torch.cuda.memory_allocated(i),torch.cuda.memory_allocated(i)/1024**3)
#         print('Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB', torch.cuda.memory_reserved(i),torch.cuda.memory_reserved(i)/1024**3)
#         print("---------------------------------------------------------------")

a = torch.rand((3,4))
b = torch.rand_like(a).ge(0.5)

print(a,"\n", b)
print(a[b])
print(a.masked_select(b))

print(torch.where(b==True,1,0))

from enum import IntEnum
class edge_types(IntEnum):
    spatial_edges = 0
    temporal_edges = 1

y = torch.LongTensor(4,1).random_() % 2
print("y:", y)
y_onehot = torch.FloatTensor(4, 2)
y_onehot.zero_()
y_onehot.scatter_(1,y,1)
print("y_onehot:", y_onehot)

tensor([[0.5638, 0.4236, 0.3750, 0.8855],
        [0.2826, 0.2367, 0.2039, 0.9313],
        [0.6879, 0.6946, 0.6231, 0.0440]]) 
 tensor([[ True, False,  True,  True],
        [False, False, False, False],
        [ True,  True,  True, False]])
tensor([0.5638, 0.3750, 0.8855, 0.6879, 0.6946, 0.6231])
tensor([0.5638, 0.3750, 0.8855, 0.6879, 0.6946, 0.6231])
tensor([[1, 0, 1, 1],
        [0, 0, 0, 0],
        [1, 1, 1, 0]])
y: tensor([[1],
        [0],
        [0],
        [1]])
y_onehot: tensor([[0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.]])


In [17]:
#Create List of Graph objects
from datasets.nuscenes_mot_graph import NuscenesMotGraph
#______________________________________________________________#
# Decide if only first scene should be computed
only_first_scene = True
scene_token0, sample_token0= sample_dict[0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    for i in range(torch.cuda.device_count()):
        if torch.cuda.get_device_name(i) == "GeForce RTX 2080":
            device = torch.device(i) # RTX 2080 8GB
print("Device:", device)
#_______________________________________________________________#

MotGraphList= []
for sample_key in sample_dict:
    scene_token_current, sample_token_current= sample_dict[sample_key]
    if(only_first_scene):
        if(scene_token0 == scene_token_current):
            object = NuscenesMotGraph(nuscenes_handle = nusc,
                        start_frame=sample_token_current,
                        max_frame_dist = 3, 
                        filterBoxes_categoryQuery='vehicle.car',
                        device= device)
            is_possible2construct = object.is_possible2construct
            if is_possible2construct:
                object.construct_graph_object()
                # object.assign_edge_labels_one_hot()
                object.assign_edge_labels(label_type='multiclass')
                # object.assign_edge_labels(label_type='binary')
                MotGraphList.append(object)
    else:
        object = NuscenesMotGraph(nuscenes_handle = nusc,
                        start_frame=sample_token_current,
                        max_frame_dist = 3,  
                        filterBoxes_categoryQuery='vehicle.car',device= device)
        is_possible2construct = object.is_possible2construct
        if is_possible2construct:
            object.construct_graph_object()
            object.assign_edge_labels(label_type='binary')
            MotGraphList.append(object)
            

Device: cuda:1


KeyError: 'relative_vectors'

In [ ]:
# # Input
# print("----------------------------------------\n")
# print(MotGraphList[0].graph_obj.keys)
# print("----------------------------------------\n")
# print("Node features: ", MotGraphList[0].graph_obj['x'])
# print("----------------------------------------\n")
# print("Edge indices", MotGraphList[0].graph_obj['edge_index'])
# print("----------------------------------------\n")
# print("Edge attributes (only temporal until now)", MotGraphList[0].graph_obj['edge_attr'])

In [ ]:
# #Output
# from groundtruth_generation.nuscenes_create_gt import edge_label_classes

# print("edge_label_classes.different_instance:",edge_label_classes.different_instance)
# print("edge_label_classes.same_instance:",edge_label_classes.same_instance)
# print("edge_label_classes.new_instance:",edge_label_classes.new_instance)
# print("Edge labels: one hot encoding",MotGraphList[0].graph_obj["edge_labels"])

In [ ]:
print("----------------------------------------\n")
print(len(MotGraphList))
print("----------------------------------------\n")
print(MotGraphList[0])
print("----------------------------------------\n")
print(MotGraphList[0].graph_obj)
print("----------------------------------------\n")
print(len(MotGraphList[0].graph_obj))
print("----------------------------------------\n")
print(MotGraphList[0].graph_obj.keys)
print("----------------------------------------\n")
print("is_undirected() : ",MotGraphList[0].graph_obj.is_undirected())
print("----------------------------------------\n")
print(MotGraphList[0].graph_obj["x"][0:5])

print("----------------------------------------\n")
print('graph_dataframe["boxes_dict"][1][0] \n',MotGraphList[0].graph_dataframe["boxes_dict"][1][0])
print("----------------------------------------\n")
box_list, center_list = MotGraphList[0].graph_dataframe["centers_dict"][1]
print("box_list: \n",box_list[0], 'center_list: \n', center_list[0])
print("----------------------------------------\n")
box_list, center_list = MotGraphList[0].graph_dataframe["centers_dict"][1]
print("is same memory ?: \n", MotGraphList[0].graph_dataframe["boxes_dict"][1] is box_list )
print("----------------------------------------\n")
for i in range(3):
    print(MotGraphList[0].graph_dataframe["boxes_list_all"][i] )
    print(MotGraphList[0].graph_dataframe["centers_list_all"][i])
    print("----------------------------------------\n")
print("----------------------------------------\n")
for i in range(3):
    print(MotGraphList[0].graph_dataframe["boxes_list_all"][-i] )
    print(MotGraphList[0].graph_dataframe["centers_list_all"][-i])
    print("----------------------------------------\n")
print("----------------------------------------\n")
print("Edge Labels: ",MotGraphList[0].graph_obj["edge_labels"])
print("----------------------------------------\n")
print("Edge Labels: ",MotGraphList[-1].graph_obj["edge_labels"])
print("----------------------------------------\n")

print("shape: ",MotGraphList[0].graph_obj.x.shape)
print("----------------------------------------\n")
print("Edge Labels: ",MotGraphList[0].graph_obj["edge_index"])
print("----------------------------------------\n")

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(3, 16)
        self.conv2 = GCNConv(16, 2)
        self.mlp = torch.nn.Linear(
                    in_features=3, out_features=1, bias=True)

    def forward(self, data):
        x = data.x 
        edge_index = data.edge_index
        edge_feature =data.edge_attr

        # x = self.conv1(x, edge_index)
        x = F.relu(x)
        # x = self.conv2(x, edge_index)
        x = x.float()
        x = self.mlp(x)

        return F.sigmoid(x)

In [ ]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

graph_list = []
for graph in MotGraphList:
    graph_list.append(graph.graph_obj)
print(len(graph_list))
print(graph_list[:5])
train_loader = DataLoader(graph_list, batch_size=1,shuffle =False)

In [ ]:
# for data in train_loader:
    # print(data.x)
    # print(data.edge_index)
    # print(data.edge_attr)
    # print(data.num_graphs)

# a = torch.zeros(3,5).to('cuda')
# probabilities_labels = torch.empty(a.shape).uniform_(0, 1).to('cuda')
# label = torch.bernoulli(probabilities_labels).to('cuda')
# print(a)
# print(probabilities_labels)
# print(label)

In [ ]:

# def train():
#     model.train()

#     loss_all = 0
#     for data in train_loader:
#         data = data.to(device)
#         optimizer.zero_grad()
#         output = model(data)
#         label = data.y.to(device)
#         loss = crit(output, label)
#         loss.backward()
#         loss_all += data.num_graphs * loss.item()
#         optimizer.step()
#     return loss_all / len(graph_list)
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = Net().to(device)
print(next(model.parameters()).is_cuda)
# print(model.parameters())

optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.BCELoss()

print(train_loader)

loss_general = []
for epoch in range(4):
    model.train()
    loss_all = 0
    for data in train_loader:
        optimizer.zero_grad()
        output = model(data)
        #Generate random labels for testing pipeline
        # generate binary labels 
        probabilities_labels = torch.empty(output.shape).uniform_(0, 1).to(device)
        node_labels_binary = torch.bernoulli(probabilities_labels)
        loss = crit(output, node_labels_binary)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    loss_normalized =loss_all / len(graph_list)
    loss_general.append(loss_normalized)
    print('epoch',epoch,'loss_general:',loss_normalized)

In [ ]:
import tqdm